In [7]:
import pandas as pd

In [15]:
path = "./"
data = pd.read_csv(path + 'train_events_adjusted_time.csv')
old = pd.read_csv(path + 'train_events.csv')
video = pd.read_csv(path + 'video_info_v2.csv')
targets = pd.read_csv(path + 'train_targets.csv')

old = old[['viewer_uid', 'event_timestamp', 'rutube_video_id']]

In [16]:
def remove_outliers(df, columns, method='iqr', factor=1.5):
    """
    Удаляет выбросы из указанных столбцов DataFrame с использованием указанного метода.

    Параметры:
    - df (pd.DataFrame): исходный DataFrame
    - columns (list): список столбцов для очистки от выбросов
    - method (str): метод для удаления выбросов, может быть 'iqr' или 'std'
                    ('iqr' - интерквартильный размах, 'std' - стандартное отклонение)
    - factor (float): множитель для определения границ выбросов (по умолчанию 1.5 для IQR и 3 для std)

    Возвращает:
    - pd.DataFrame: DataFrame без выбросов в указанных столбцах
    """

    df_clean = df.copy()

    if method not in ['iqr', 'std']:
        raise ValueError("Метод должен быть 'iqr' или 'std'")

    for column in columns:
        if method == 'iqr':
            Q1 = df_clean[column].quantile(0.25)
            Q3 = df_clean[column].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - factor * IQR
            upper_bound = Q3 + factor * IQR

        elif method == 'std':
            mean = df_clean[column].mean()
            std_dev = df_clean[column].std()
            lower_bound = mean - factor * std_dev
            upper_bound = mean + factor * std_dev

        # Фильтрация DataFrame по границам выбросов
        df_clean = df_clean[(df_clean[column] >= lower_bound) & (df_clean[column] <= upper_bound)]

    return df_clean

In [17]:
merge_data = data.merge(video, how="left", on="rutube_video_id")
del video, data
merge_data = merge_data.merge(targets[['age', 'age_class', 'sex', 'viewer_uid']],
                              how="left", on="viewer_uid")

del targets

In [18]:
# Очистка выбросов
merge_data = remove_outliers(merge_data, ['total_watchtime'], method='iqr')

# Заполнение пропусков
mode_value = merge_data['ua_os'].mode()[0]
merge_data['ua_os'].fillna(mode_value, inplace=True)

mode_value = merge_data['ua_client_name'].mode()[0]
merge_data['ua_client_name'].fillna(mode_value, inplace=True)


# Удаление канала, который только зашумляет наши данные
merge_data = merge_data[merge_data['author_id'] != 1009257]
merge_data = merge_data[merge_data['author_id'] != 1043618]

C:\Users\dlevi\AppData\Local\Temp\ipykernel_4276\2359768156.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merge_data['ua_os'].fillna(mode_value, inplace=True)
C:\Users\dlevi\AppData\Local\Temp\ipykernel_4276\2359768156.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

In [19]:
old['date'] = pd.to_datetime(old['event_timestamp']).dt.day.astype(str)

print(old['date'].head())

old = old.drop(columns=['event_timestamp'])

merge_data['date'] = old['date'][merge_data.index]
del old

0    1
1    1
2    1
3    1
4    1
Name: date, dtype: object


In [20]:
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1258601 entries, 0 to 1759615
Data columns (total 18 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   Unnamed: 0       1258601 non-null  int64 
 1   region           1258601 non-null  object
 2   ua_device_type   1258601 non-null  object
 3   ua_client_type   1258601 non-null  object
 4   ua_os            1258601 non-null  object
 5   ua_client_name   1258601 non-null  object
 6   total_watchtime  1258601 non-null  int64 
 7   rutube_video_id  1258601 non-null  object
 8   viewer_uid       1258601 non-null  int64 
 9   adjusted_time    1258601 non-null  object
 10  title            1258601 non-null  object
 11  category         1258601 non-null  object
 12  duration         1258601 non-null  int64 
 13  author_id        1258601 non-null  int64 
 14  age              1258601 non-null  int64 
 15  age_class        1258601 non-null  int64 
 16  sex              1258601 non-null  object

In [22]:
merge_data.drop(columns=['Unnamed: 0'], inplace=True)
merge_data.head()

,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,adjusted_time,title,category,duration,author_id,age,age_class,sex,date
0,Chelyabinsk,desktop,browser,Windows,Yandex Browser,1883,video_133074,10067243,08:40:58,Папа с особенностями. Мужское / Женское. Выпус...,Телепередачи,2456534,1009219,20,0,female,1
1,Bashkortostan Republic,smartphone,mobile app,Android,Rutube,512,video_362960,10245341,21:33:24,Comedy Club: Мальдивы | Андрей Бебуришвили,Юмор,519211,1006760,40,2,female,1
3,Moscow,smartphone,mobile app,Android,Rutube,1521,video_161610,10029092,23:03:42,Сергей Орлов-снял дом!!!,Разное,1522069,1058671,41,3,male,1
4,Moscow,smartphone,mobile app,Android,Rutube,71,video_116245,10452976,22:48:09,Ищем сокровища в Полевском | Уральская Флоренц...,Путешествия,1249920,1020020,38,2,female,1
5,Rostov,smartphone,mobile app,Android,Rutube,60,video_61152,10013813,04:11:51,День защиты детей. Мужское / Женское. Выпуск о...,Телепередачи,2465238,1009219,44,3,female,1


In [23]:
merge_data.to_csv(path + "clear_data_times_data.csv")